# System for Distraction Detection and Monitoring

State Farm Distracted Driver Detection has divided driving behavior into 10 classes.  


| CLASS        | Description          |  
| :------------- |:-------------|  
| C0     | Safe Driving | 
| C1     | Texting - Right      |
| C2     | Talking On The Phone - Right    |
| C3     | Texting- left | 
| C4     | Talking on the Phone- left | 
| C5     | Operating The Radio   |
| C6     | Drinking| 
| C7     |  Reaching Behind     |
| C8     | Hair And Makeup   |
| C9     | Talking To Passenger   |


We will implement a deep learning model, which will predict the driver activities at the time of driving. 

# Importing libraries 

In [ ]:
import cv2
import numpy as np 
import pandas as pd 
import os
import random
import shutil
import matplotlib.pyplot as plt
# image processing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#keras models
from tensorflow.keras import layers ,models,optimizers
from keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.utils import plot_model
#resnet50
from tensorflow.keras.applications import resnet50
from tensorflow.keras.applications.imagenet_utils import preprocess_input 
#stop criteria
from tensorflow.keras.callbacks import EarlyStopping
#cnn_visualisation
from keras.preprocessing import image


# moving to working directory

In [ ]:
!cp -r /kaggle/input/state-farm-distracted-driver-detection/imgs/train ./


In [ ]:
train_dir = "/kaggle/working/train/"
valid_dir ="/kaggle/working/val/"
test_dir  = "/kaggle/working/test/" 

# defining classes

In [ ]:
activity = {   'c0' : "safe_driving",
                      'c1' : "texting-right",
                      'c2' : "talking_on_the_phone-right",
                      'c3' : "texting-left",
                      'c4' : "talking_on_the_phone-left",
                      'c5' : "operating_the_radio",
                      'c6' : "drinking",
                      'c7' : "reaching_behind",
                      'c8' : "hair-and-makeup",
                      'c9' : "talking_to_passenger"}
activity.keys()

# prepare data
**contact each image to its path and label it**

In [ ]:
for file in os.listdir(train_dir):
    shutil.move(os.path.join(train_dir,file), os.path.join(train_dir,activity[f'{file}']))

In [ ]:
for file in os.listdir(train_dir):
    os.makedirs(valid_dir + '/' + file, exist_ok=True)
    os.makedirs(test_dir + '/' + file, exist_ok=True)    
    train_dir_img = train_dir + file
    file_len = len([sample for sample in os.listdir(train_dir_img)])
    print(file_len)   # no of sampels for each class
    
    for sample in random.sample(os.listdir(train_dir_img) , int(float(0.1) * file_len)):
        shutil.move(train_dir_img + '/' + sample, valid_dir + file)# Moving source to newly created destination 
    for sample in random.sample(os.listdir(train_dir_img) , int(float(0.1) * file_len)):
        shutil.move(train_dir_img + '/' + sample, test_dir + file)

# plot one image from each class

In [ ]:
plt.figure(figsize = (12, 20))
image_count = 1
BASE_URL = '../input/state-farm-distracted-driver-detection/imgs/train/'
for directory in os.listdir(BASE_URL):
    if directory[0] != '.':
        for i, file in enumerate(os.listdir(BASE_URL + directory)):
            if i == 1:
                break
            else:
                fig = plt.subplot(5, 2, image_count)
                image_count += 1
                image =cv2.imread(BASE_URL + directory + '/' + file)
                img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                plt.imshow(img)
                plt.title(activity[directory])

# **RESIZE IMAGES**

# ImageDataGenerator usually used with rescaling factor 1./255 to rescale the initial values from 0 to 255 to 0 to 1 instead.

In [ ]:
datagen = ImageDataGenerator(rescale=1 / 255.0)
# ImageDataGenerator and usually it is used with rescaling factor 1./255 to rescale the initial values from 0 to 255 to 0 to 1 instead.

# ImageDataGenerator
flow_from_directory (directory), Description:Takes the path to a directory, and generates batches of augmented/normalized data.

https://www.datasnips.com/blog/2021/8/30/Image-Augmentation-Using-Keras-ImageDataGenerator-with-Tensorflow/

In [ ]:
batch_size = 32
train_batches =datagen.flow_from_directory(directory = train_dir,shuffle = True,batch_size = batch_size)
val_batches =datagen.flow_from_directory(directory = valid_dir,shuffle = True, batch_size = batch_size)
test_batches =datagen.flow_from_directory(directory= test_dir,shuffle = False,batch_size = 1)

# Dense model

In [ ]:
network = models.Sequential()
network.add(layers.Flatten(input_shape=(256,256,3)))
network.add(layers.Dense(1024,activation = 'relu',name = 'input'))
network.add(layers.BatchNormalization())
#Batch normalization applies a transformation that maintains the mean 
#output close to 0 and the output standard deviation close to 1.
network.add(layers.Dense(512,activation = 'relu',name = 'l1'))
network.add(layers.BatchNormalization())
network.add(layers.Dense(256,activation = 'relu',name = 'l2'))
network.add(layers.BatchNormalization())
network.add(layers.Dense(10,activation = 'softmax',name = 'output'))

In [ ]:
plot_model(network)

In [ ]:
network.compile(optimizer='Adam',
                loss='categorical_crossentropy',
                metrics=['acc'])

In [ ]:
stop_criteria = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)
dense_model=network.fit(x = train_batches,
          steps_per_epoch=350,
          epochs=15,
          validation_data = val_batches,
          validation_steps= 50
          , callbacks=[stop_criteria]
                       )

In [ ]:
plt.plot(dense_model.history['acc'])  # blue line
plt.plot(dense_model.history['val_acc']) #orange line
plt.title('model accuracy')
plt.ylabel('accuracy')

In [ ]:
dense_scores= network.evaluate(test_batches)
print("Accuracy: %.2f%%" % (dense_scores[1]*100))

# DATA Augmentation

In [ ]:
train_datagen = ImageDataGenerator( rescale=1 / 255.0,
                                    zoom_range=0.05,
                                    width_shift_range=0.05,
                                    height_shift_range=0.05,
                                    shear_range=0.05,
                                    fill_mode="nearest")

# CNN WITH DATA Augmentation

In [ ]:
cnn_model = models.Sequential()
cnn_model.add(layers.Conv2D(32,(3,3),activation = 'relu',name = 'Conv_',input_shape = (256,256,3)))
cnn_model.add(layers.Conv2D(32,(3,3),activation = 'relu',name = 'Conv_2',padding = 'same'))
cnn_model.add(layers.Conv2D(32,(3,3),activation = 'relu',name = 'Conv_3',padding = 'same'))
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.MaxPooling2D((2,2),name = 'max_1'))
cnn_model.add(layers.Conv2D(64,(3,3),activation = 'relu',name = 'Conv_4',padding='same'))
cnn_model.add(layers.Conv2D(64,(3,3),activation = 'relu',name = 'Conv_5',padding='same'))
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.MaxPooling2D((2,2),name = 'max_2'))

cnn_model.add(layers.Conv2D(128,(3,3),activation='relu'))
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.MaxPooling2D((2,2)))
cnn_model.add(layers.Conv2D(128,(3,3),activation='relu'))
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.Flatten())
cnn_model.add(layers.Dense(512,activation = 'relu',name = 'L1',))
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.Dense(256,activation = 'relu',name = 'L2'))
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.Dense(256,activation = 'relu',name = 'L3'))
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.Dense(128,activation = 'relu' ,name ='L4'))
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.Dense(10,activation = 'softmax',name = 'output'))

In [ ]:
plot_model(cnn_model,"model.png",show_shapes=True,show_layer_names=True)

In [ ]:
cnn_model.compile(optimizer = 'adam' , loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
stop_criteria = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)
cnn_model_history = cnn_model.fit(x = train_batches,
          steps_per_epoch=250,
          epochs=15,
          validation_data = val_batches,
          validation_steps= 50,
          callbacks=[stop_criteria])

# None of the MLIR Optimization Passes are enabled is a bit misleading as it refers to very particular workflow. 
# But it is benign and has no effect - it just means a user didn’t opt in to a specific pass (which is not enabled by default), 
# so it doesn’t indicate any error and was rather used as signal for developers.

In [ ]:
acc = cnn_model_history.history['acc']
val_acc = cnn_model_history.history['val_acc']

loss = cnn_model_history.history['loss']
val_loss = cnn_model_history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b' ,label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
cnn_scores = cnn_model.evaluate(test_batches)
print("Accuracy: %.2f%%" % (cnn_scores[1]*100))

# Transfer learning with resnet50

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function= resnet50.preprocess_input,
                                                zoom_range=0.05,
                                                width_shift_range=0.05,
                                                height_shift_range=0.05,
                                                shear_range=0.05,
                                                fill_mode="nearest")

resnet_datagen = ImageDataGenerator(preprocessing_function= resnet50.preprocess_input)

# CNN WITH RESNET

In [ ]:
conv_model = resnet50.ResNet50(weights='imagenet',include_top=False,input_shape = (256,256,3))

for layer in conv_model.layers[:-3]:
    layer.trainable=False  #The role of the embedding layer is to map a category into a dense space in a way that is useful for the task

resnet_model = models.Sequential()
resnet_model.add(conv_model)
resnet_model.add(layers.Flatten())

cnn_model.add(layers.Dense(512,activation = 'relu',))
cnn_model.add(layers.BatchNormalization())

resnet_model.add(layers.Dense(256,activation = 'relu'))
cnn_model.add(layers.BatchNormalization())

resnet_model.add(layers.Dense(128,activation = 'relu'))
cnn_model.add(layers.BatchNormalization())

resnet_model.add(layers.Dense(10,activation = 'softmax',name = 'output'))  # MULTICLASSES

In [ ]:
plot_model(resnet_model,"model.png",show_shapes=True,show_layer_names=True)

In [ ]:
resnet_model.compile(optimizer = optimizers.Adam(learning_rate=.0001) ,
              loss='categorical_crossentropy',
              metrics=['acc'])

In [ ]:
stop_criteria = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)
resnet_model_history = resnet_model.fit(x = train_batches,
          steps_per_epoch=250,
          epochs=15,
          validation_data = val_batches,
          validation_steps= 50,
          callbacks=[stop_criteria])

In [ ]:
acc = resnet_model_history.history['acc']
val_acc = resnet_model_history.history['val_acc']

loss = resnet_model_history.history['loss']
val_loss = resnet_model_history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b' ,label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
resnet_scores= resnet_model.evaluate(test_batches)
print("Accuracy: %.2f%%" % (resnet_scores[1]*100))

# CNN Visualization

In [ ]:
# Pre-processing the image
images, labels  = next(train_batches)
img_tensor = images[1]
img_tensor = np.expand_dims(img_tensor, axis=0)

# Print image tensor shape
print(img_tensor.shape)

# Print image
plt.imshow(img_tensor[0])
plt.show()

In [ ]:
layer_outputs = [layer.output for layer in cnn_model.layers[:10]]
activation_model = models.Model(inputs=cnn_model.input, outputs=layer_outputs)
activations = activation_model.predict(img_tensor)

In [ ]:
layer_names = []
for layer in resnet_model.layers[:10]:
    layer_names.append(layer.name)

images_per_row = 16

# Now let's display our feature maps
for layer_name, layer_activation in zip(layer_names, activations):
    # This is the number of features in the feature map
    n_features = layer_activation.shape[-1]

    # The feature map has shape (1, size, size, n_features)
    size = layer_activation.shape[1]

    # We will tile the activation channels in this matrix
    n_cols = n_features // images_per_row
    display_grid = np.zeros((size * n_cols, images_per_row * size))

    # We'll tile each filter into this big horizontal grid
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0,
                                             :, :,
                                             col * images_per_row + row]
            # Post-process the feature to make it visually palatable
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size,
                         row * size : (row + 1) * size] = channel_image

    # Display the grid
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

plt.show()

# ACCURACY OF MODELS USED

# models evaluation

In [ ]:
# dense_scores
acc1=dense_model.history['acc'][-1]
vacc1=dense_model.history['val_acc'][-1]
# cnn_scores
acc2=cnn_model_history.history['acc'][-1]
vacc2=cnn_model_history.history['val_acc'][-1]
# resnet_scores
acc3=resnet_model_history.history['acc'][-1]
vacc3=resnet_model_history.history['val_acc'][-1]

In [ ]:
results = pd.DataFrame([["DENSE model",acc1,vacc1,dense_scores[1]*100],
                       ["CNN_data_augmentation",acc2,vacc2,cnn_scores[1]*100],
                       ["CNN_RESNET model",acc3,vacc3,resnet_scores[1]*100]],
                       columns = ["Model","Training Accuracy %","Validation Accuracy %","Test Evaluation %"]).sort_values(by="Test Evaluation %",ascending=False)
results.style.background_gradient(cmap='BuPu')